In [5]:
import os

os.getcwd()

'/workspaces/GodTierDevStatus/🔔'

In [3]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
import plotly.express as px
from ..code.chris_functions import chris_simple_make_blobs, latex_helper

ImportError: attempted relative import with no known parent package

# **The Elements of Statistical Learning: Data Mining, Inference, and Prediction, 2nd ed., Hastie, Tibshirani, Friedman**

Christopher La Valle

## **2 - Overview of Supervised Learning**

Given a set of variables that might be denoted as *inputs*, which are measured or preset, and these have some influence on one or more *outputs*. The goal is to use the inputs to predict the values of the outputs. This exrcise is called *supervised learning*.

We can loosely state the learning task as afolows: given the value of an input vector $X$, make a good prediction of the output $Y$, denoted $\hat{Y}$.

We need data to construct prediction rules, often a lot of it. We thus suppose we have available a set of measurements $(x_i,y_i)$ or $(x_i,y_i),\ i=1,\ldots,N$, known as the *training data*, with which to construct our prediction rule.

### **2.3 - Two Simple Approaches to Prediction: Least Squares and Nearest Neighbors**

The linear model makes huge assumptions about structure and yields stable but possibly innacurate predictions. The method of $k$-nearest neighbors makes very mild structural assumptions: its predictions are often accurate but can be unstable.

#### **2.3.1 - Linear Models and Least Squares**

Given a vector of inputs $X^T=(X_1,X_2,\ldots,X_p)$, we predict the output $Y$ via the model

$$\hat{Y}=\hat{\beta}_0+\sum_{j=1}^pX_j\hat{\beta}_j$$

where $\hat{\beta}_0$ is the intercept, also known as the *bias* in machine learning. Often it is convenient to include the constant variable $1$ in X, include $\hat{\beta}_0$ in the vector of coefficients $\hat{\beta}$, and then write the linear model in vector form as an inner product

$$\hat{Y}=X^T\hat{\beta}$$

Here we are modeling a single output, so $\hat{Y}$ is a scalar; in general $\hat{Y}$ can be a $K$-vector, in which case $\beta$ would be a $p\times K$ matrix of coefficients. In the $(p+1)$-dimensional input-output space, $(X,\hat{Y})$ represents a hyperplane. If the constant is included in $X$, then the hyperplane includes the origin and is a subspace; if not, it is an affine set cutting the $Y$-axis at the point $(0,\hat{\beta}_0)$. 

Viewed as a function over the $p$-dimensional input space, $f(X)=X^T\beta$ is linear, and the gradient $f'(X)=\beta$ is a vector in input space that points in the steepest uphill direction.

A popular approach to fitting the model the a set of training data is by *least squares*. In this approach, we pick the coefficients $\beta$ that minimize the residual sum of squares

$$RSS(\beta)=\sum_{i=1}^n(y_i-x_i^T\beta)^2$$

$RSS(\beta)$ is a quadratic function of these parameters, and hence its maximum always exists, but may not be unique. The solution is easiest to characterize in matrix notation.

$$RSS(\beta)=(\mathbf{y}-\mathbf{X}\beta)^T(\mathbf{y}-\mathbf{X}\beta)$$

where $\mathbf{X}$ is an $N\times p$ matrix with each row an input vector, and $\mathbf{y}$ is an $N$-vector of the outputs in the training set. Differentiating w.r.t $\beta$ we get the *normal equations*

$$\mathbf{X}^T(\mathbf{y}-\mathbf{X}\beta)=0$$

If $\mathbf{X^T}$ is nonsingular, then the unique solution is given by

$$\hat{\beta}=(\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}$$

and the fitted value at the $i$th input $x_i$ is $\hat{y}_i=\hat{y}(x_i)=x_i^T\hat{\beta}$. At an arbitrary input $x_0$ the prediction is $\hat{y}(x_0)=x_0^T\hat{\beta}$. The entire fitted surface is characterized by the $p$ parameters $\hat{\beta}$.


In [2]:
n_samples = 1_000
x = np.random.normal(loc=0, scale=0.5, size=(n_samples, ))
y = 4 * x + 8 + np.random.normal(loc=0, scale=1, size=(n_samples, ))

lg = LinearRegression()
lg.fit(x.reshape(-1, 1), y.reshape(-1, 1))
l = np.linspace(x.min(), x.max())
g = lg.predict(l.reshape(-1, 1)).reshape(len(l), )

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=x,
        y=y,
        mode='markers',
        marker=dict(
            color='orange',
            line=dict(
                color='red',
                width=1,
            )
        )
    )
)


fig.add_trace(
    go.Scatter(
        x=l,
        y=g,
        mode='lines',
        line=dict(
            color='blue'
        )
        
    )
)


fig.update_layout(
    xaxis_title='X',
    yaxis_title='y',
    showlegend=False,

)


In [3]:
n_samples = 500
n_centers = 2
spread = 5

T, means = chris_simple_make_blobs(n_samples, n_centers, spread=spread)


lg = LinearRegression()
lg.fit(T[['x_1', 'x_2']], T['y'])
l = np.linspace(T['x_1'].min(), T['x_1'].max(), 100)
g = (0.5 - (lg.coef_[0] * l + lg.intercept_)) / lg.coef_[1]

T['y_hat'] = lg.predict(T[['x_1', 'x_2']])

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=T['x_1'],
        y=T['x_2'],
        mode='markers',
        marker=dict(
            size=7,
            color=T['y'].apply(lambda x: '#FEB270' if x else '#7086FE'),
            line=dict(
                color=T['y_hat'].apply(lambda x: 1 if x > 0.5 else 0).apply(lambda x: '#F87604' if x else '#002AFC'),
                width=2
            )
        )
    )
)


fig.add_trace(
    go.Scatter(
        x=l,
        y=g,
        mode='lines',
        line=dict(
            color='red'
        )
        
    )
)


fig.update_layout(
    showlegend=False,

)



NameError: name 'chris_simple_make_blobs' is not defined

Nearest neighbors uses those observations in the training set $\Tau$ that are closest in input space to $x$ to form $\hat{Y}$. Specifically, the $k$-nearest neighbors fit for $\hat{Y}$ is defined as follows 

$$\hat{Y}=\frac{1}{k}\sum_{x_i\in N_k(x)}y_i$$

where $N_k(x)$ is the neighborhood of $x$ defined by the $k$ closest points $x_i$ in the training sample. Closeness implies a metric, which for the moment we assume is Euclidean distance.

In [4]:
n_samples = 500
n_centers = 2
spread = 5
n_neighbors = 20

T, means = chris_simple_make_blobs(n_samples, n_centers, spread=spread)


knn = KNeighborsClassifier(n_neighbors=n_neighbors)
knn.fit(T[['x_1', 'x_2']], T['y'])
T['y_hat'] = pd.Series(knn.predict(T[['x_1', 'x_2']]))


fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=T['x_1'],
        y=T['x_2'],
        mode='markers',
        marker=dict(
            size=7,
            color=T['y'].apply(lambda x: '#FEB270' if x else '#7086FE'),
            line=dict(
                color=pd.Series(T['y_hat'].apply(lambda x: '#F87604' if x else '#002AFC')),
                width=2
            )
        )
    )
)


fig.update_layout(
    showlegend=False,
)




NameError: name 'chris_simple_make_blobs' is not defined

$k$-nearest-neighbors may misclassify fewer obsrvations from the training data, but an independent test set would give us more satisfactory means for judging the model's accuracy. 

It appears that $k$-nearest-neighbor fits have a single parameter, the number of neighbors $k$, compared to the $p$ parameers in least-squares fits. Although this is the case, we will see that the *effective* number of parameters of $k$-nearest neighbors is $N/k$ and is generally bigger than $p$, and decreases with increasing $k$. To get an idea of why, note that if the neighborhoods were nonoverlapping, there would be $N/k$ neighborhoods and we would fit one parameter (a mean) in each neighborhood.

# More here

### **2.4 - Statistical Decision Theory**

Let $X\in\mathbb{R}^p$ denote a real valued random input vector, and $Y\in\mathbb{R}$ a real value random output variable, with joint distribution $Pr(X,Y)$. We seek a function $f(X)$ for predicting $Y$ given the values of the input $X$. This theory requires a *loss function* $L(Y,f(X))$ for penalizing errors in prediction, and by far the most common and convenient is *squared error loss*: $L(Y,f(X))=(Y-f(X))^2$. 

This leads to a criterion for choosing $f$,

$$
\begin{align*}
EPE(f)&=E(Y-f(X))^2\tag{2.9}\\
&=\int[y-f(x)]^2Pr(dx,dy)\tag{2.10}
\end{align*}
$$

the expected (squared) prediction error. By conditioning on $X$, we can write $EPE$ as

$$EPE(f)=E_XE_{Y|X}([Y-f(X)]^2|X)\tag{2.11}$$

and we see that it suffices to minimize $EPE$ pointwise:

$$f(x)={\arg\min}_cE_{Y|X}([Y-c]^2|X=x)\tag{2.12}$$

The solution is

$$f(x)=E(Y|X=x)\tag{2.13}$$

the conditional expection, alson known as the *regression function*. Thus the best prediction of $Y$ at any point $X=x$ is the conditional mean, when best is measured by average squared error.

The nearest-neighbor methods attempt to directly implement this recipe using the training data. At each point $x$, we might ask for the average of all thsoe $y_i$'s with input $x_i=x$. Since there is typically at most one observation at any point $x$, we settle for 

$$\hat{f}(x)=\text{Ave}(y_i|x_i\in N_k(X))$$

where $N_k(x)$ is the neighborhood containing the $k$ points in $\mathcal{T}$ closest to $x$. Two approximation are happening here:

 - expectation is approximated by averaging over sample data;
 - conditioning at a point is relaxed to conditioning on some region "close" to the the target point.

For large training sample size $N$, the points in the neighborhood are likely to be close to $x$, and as $k$ gets large the average will gt more stable. In fact, under mild regularity conditions on the joint probability distribution $\text{Pr}(X,Y)$, one can show that as $N,k\rightarrow\infty$ such that $k/N\rightarrow0$, $\hat{f}(x)\rightarrow E(Y|X=x)$.





For linear regression, one assumes that the regression function $f(x)$ is approximately linear in its arguments:

$$f(x)\approx x^T\beta\tag{2.15}$$

This is a model-based approach--we specify a model for the regression function. Plugging this linea rmodel for $f(x)$ into $\text{EPE}$ (2.9) and differentiating we can solve for $\beta$ theoretically:

$$\beta=[E(XX^T)]^{-1}E(XY)\tag{2.16}$$

Note we have not conditioned on $X$; rather we have used our knowledge of the functional relationship to pool over values of $X$. The least squares solution (2.6) amounts to replacing the expectation in (2.16) by averages over the training data.

So both $k$-nearest neighbors and least squares end up approximating conditional expectations by averages. But they differ dramatically in terms of model assumptions:

 - Least squares assumes $f(x)$ is well approximated by a globally linear function.
 - $k$-nearest neighbors assumes $f(x)$ is well approximated by a locally constant function.

Many of the more modern techniques described in this book are model based, although far more flexible than the rigid linear model. For example, additive models assume that 

$$f(X)=\sum_{j=1}^pf_j(X_j)\tag{2.17}$$

This retains the additivity of the linear model, but each coordinate function $f_j$ is arbitrary. It turns out that the optimal estiamte for the additive model uses techniques such as $k$-nearest neighbors to approximate univariate conditional expectations simultaneously for each of the coordinate functions. Thus the problems of estimating a conditional expectation in high dimensions are swept away in this case by imposing some (often unrealistic) model assumptions, in this case additivity.

What happens if we replace the $L_2$ loss function with the $L_1: E|Y-f(X)|$? The solution in this case is the conditional mean

$$\hat{f}(x)=\text{median}(Y|X=x)\tag{2.18}$$

which is a different measure of location, and its estimates are more robust than those for the conditional mean. $L_1$ criteria have discontinuities in their derivatives, which have hindered their widespread use. 

What do we do when the output is a categorical variable $G$? The same paradigm works here, except we need a different loss function for penalizing predicition errors. An estimate $\hat{G}$ will assume values in $\mathcal{G}$, the set of possible classes. Our loss function can be represented by a $K\times K$ matrix $\mathbf{L}$, where $K=card(\mathcal{G})$. $\mathbf{L}$ will be zero on the diagonal and nonnegative elsewhere, where $L(k,\mathcal{l})$ is the price paid for classifying an observation



### **2.5 - Local Methods in High Dimensions**

We have examined two learning techniques for prediction so far: the stable but biased linear model and the less stable but apparently less biases class of $k$-nearest neighbor estimates. It would seem that with a reasonably large set of training data, we could always approximate the theoretically optimal conditional expectation by $k$-nearest neighbor averaging, since we should be able to find a fairly large neighborhood

## 3 - **Linear Methods for Regression**

### **3.1 - Introduction**

A linear model assumes that the regression function $E(Y|X)$ is linear in the inputs $X_1,\ldots,X_p$. They are simple and often provide adequate and interpretable description of how the inputs affect the output. For prediction urposes they can sometimes outperform fancier nonlinerar models, especially in situations with small numbers of training cases, low singal-to-noise rati or sparse data. Finally, linear methods can be applied to transformations of thei nputs and this considerably expands their scope.

### **3.2 - Linear Regression Models and Least Squares**

We have an input vector $X^T=(X_1,X_2,\ldots,X_p)$, and want to predict a real-valued output $Y$. The linear regression model has the form

$$f(X)=\beta_0+\sum_{j=1}^pX_j\beta_j\tag{3.1}$$

The linear model either assumes that the regression function $E(Y|X)$ is linear, or that the linear model is a reasonable approximation. Here the $\beta_j$'s are unknown parameters or coefficients, and the variables $X_j4 can come from different sources:

 - quantitative inputs;
 - transformations of quantitative inputs, such as log, square-root, or square;
 - basis expansion, such as $X_2=X_1^2,X_3=X_1^3$, leading to a polynomial representation;
 - numeric or "dummy" coding of the levels of qualitative inputs. 
 - interactions between variables, for example, $X_3=X_1\cdot X_2$.

No matter the source of the $X_j$, the model is linear in the parameters.

Typically we have a set of training data $(x_1,y_1)\ldots(x_N,y_N)$ from which to estimate the parameters $\beta$. Each $x_i=(x_{i1},x_{i2},\ldots,x_{ip})^T$ is a vector of feature emasruements for the $i$-th case. The moset popular estimation method is *least squares*, in which we pick coefficients $\beta=(\beta_0,\beta_1,\ldots,\beta_p)^T$ to minimize the residual sum of squares

$$\begin{align*}
\text{RSS}(\beta)&=&\sum_{i=1}^N(y_i-f(x_i))^2\\
&=&\sum_{i=1}^N\left(y_i-\beta_0-\sum_{j=1}^px_{ij}\beta_j\right)^2\tag{3.2}
\end{align*}$$

From a statistical point of view, this criterion is reasonable if the training observations $(x_i,y_i)$ represent individual random draws from their population. Even if the $x_i$'s were not drawn randomly, the criterion is still valid if the $y_i$'s are conditionally independent given the inputs $x_i$. Note that (3.2) makes no assumpitons about the validitiy of (3.1); it simply finds the best linear fit to the data. Least squares fitting is intuitively satisfying no matter how the data arise; the criterion measures the average lack of fit.

Denote by $\mathbf{X}$ the $N\times(p+1)$ matrix with each row an input vector (with a $1$ in the first position), and similarly let $\mathbf{y}$ bet he $N$-vector of outputs in the training set. Then we can write the residual sum-of-squares as

$$\text{RSS}(\beta)=(\mathbf{y}-\mathbf{X}\beta)^T(\mathbf{y}-\mathbf{X}\beta)\tag{3.3}$$

This is a quadratic function in the $p+1$ parameters. Differentiating with respect to $\beta$ we obtain

$$
\begin{align*}
\frac{\partial\text{RSS}}{\partial\beta}&=-2\mathbf{X}^T(\mathbf{y}-\mathbf{X}\beta)\\
\frac{\partial^2\text{RSS}}{\partial\beta\partial\beta^T}&=2\mathbf{X}^T\mathbf{X}
\end{align*}\tag{3.4}$$

Assuming (for the moment) that $\mathbf{X}$ has full column rank, and hence $X^TX$ is positive definite, we set the first derivative to zero

$$\mathbf{X}^T(\mathbf{y}-\mathbf{X}\beta)=0\tag{3.5}$$

to obtain the unique solution

$$\hat{\beta}=(\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}\tag{3.6}$$

The predicted values at an input vector $x_0$ are given by $\hat{f}(x_0)=(1:x_0)^T\hat{\beta}; the fitted values at the training inputs are 

$$\hat{\mathbf{y}}=\mathbf{X}\hat{\beta}=\mathbf{X}(\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T\mathbf{y}\tag{3.7}$$

where $\hat{y}_i=\hat{f}(x_i)$. The matrix $\mathbf{H}=\mathbf{X}(\mathbf{X}^T\mathbf{X})^{-1}\mathbf{X}^T$ appearing in equation (3.7) is sometimes called the "hat" matrix because it puts the hat on $\mathbf{y}$.

In [5]:

N = 1000
p = 1
beta_0 = 4
beta_1 = 3

X = np.random.normal(loc=50, scale=50, size=(N, p))
y = 3 * X + beta_0 + np.random.normal(loc=0, scale=30, size=(N, 1))
X = np.hstack((np.ones((N, 1)), X)) # Add intercept

beta_hat = np.linalg.inv(X.T @ X) @ X.T @ y

x = np.linspace(X[:, 1].min(), X[:, 1].max(), 100).reshape((100, 1))
x = np.hstack((np.ones((100, 1)), x))
pred = x @ beta_hat


fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=X[:, 1],
        y=y[:, 0],
        mode='markers'
    )
)

fig.add_trace(
    go.Scatter(
        x=x[:, 1],
        y=pred[:, 0],
        mode='lines'
    )
)

## **9 - Additive Models, Trees, and Related Methods**

In the regression setting, a generalized additive model has the form

$$E(Y|X_1,X_2,\ldots,X_p)=\alpha+f_1(X_1)+f_2(X_2)+\ldots+f_p(X_p)$$

where $X_1,\ldots,X_p$ are predictors, $Y$ is the outcome, and the $f_j$'s are unspecified smooth ("nonparametric") functions. Our approach here is to fit each function using a scatterplot smoother (e.g. a cubic smoothing spline or kernel smoother), and provide an algorithm for simultaneously estimating all $p$ functions.


## **11 - Neural Networks**

### **11.1 - Introduction**

In this chapter we describe a class of learning methods that was developed seperately in different fields--statistics and artificial intelligence--based on essentially identical methods. The central idea is to extract linear combinations of the inputs as derived features, and then model the target as a nonlinear function of these function. The results is a powerful learning method, with widespread applications in many fields.

### **11.2 - Projection Pursuit Regression**

Assume we have an input vector $X$ with $p$ components, and a target $Y$. Let $\omega_m$, $m=1,2,\ldots,M$, be unit $p$-vectors of unknown parameters. The projection pursuit regression (PPR) model has the form

$$f(X)=\sum_{m=1}^Mg_m(\omega_m^TX)\tag{11.1}$$

This is an additive model, but in the derived features $V_m=\omega_m^TX$ rather than the inputs themselves. The functions $g_m$ are  unspecified and are estimated along with the directions $\omega_m$ using some flexible smoothing method.

The function $g_m(\omega_m^TX)$ is called a *ridge regression* in $\mathbb{R}^p$. It varies only in the direction defined by the vector $\omega_m$. The scalar variable $V_m=\omega_m^TX$ is the projection of $X$ onto the unit vector $\omega_m$, and we seek $\omega_m$ so that the model fits well, hence the name "projection pursuit."

The PPR model (11.1) is very general, since the operation of forming nonlinear functions of linear combinations generates a surprisingly large class of models. 

In fact, if $M$ is taken arbitrarily large, for appropriate choice of $g_m$ the PPR model can approximate any continuous function in $\mathbb{R}^p$ arbitrarily well. Such a class of models is called a *universal approximator*. However, this generality comes at a price. Interpretation of the fitted model is usually difficult, because each input enters into the model in a complex and multifaceted way. As a result, the PPR model is moset useful for prediction, and not very useful for producing an understandable model for the data. The $M=1$ model, known as the *single index model* in econometrics, is an exception. It is slightly more genreal than the linear regression model, and offers a similar interpretation.

How do we fit a PPR model, given training data $(x_i,y_i)$ $i=1,2,\ldots,N$? We seek the approximate minimizers of the error function

$$\sum_{i=1}^N\left[y_i-\sum_{m=1}^Mg_m(\omega_m^Tx_i)^2\right]^2\tag{11.2}$$

over functions $g_m$ and direction vectors $\omega_m$, $m=1,2,\ldots,M$. As in other smoothing problems, we need either explicityly or implicitly to impose complexity constraints on the $g_m$, to avoid overfit solutions.

Consider just one term ($M=1$, and drop the subscript). Given the direction vector $\omega$, we form the derived variables $v_i=\omega_Tx_i$. Then we have a one-dimensional smoothing problem, and we can apply any scatterplot smoother, such as a smoothing spline, to obtain an estimate of $g$.

On the other hand, given $g$, we want to minimize (11.2) over $\omega$. A Gauss-Newton search is convenient for this task. This is a quasi-Newton method, in which the part of the Hessian involving the second derivative of $g$ is discarded. It can be simply derived as follows. Let $\omega_{\text{old}}$ be the current estimate for $\omega$. We write

$$g(\omega^Tx_i)\approx g(\omega_{\text{old}}^Tx_i)+g'(\omega_{\text{old}}^Tx_i)(\omega-\omega_{\text{old}})^Tx_i\tag{11.3}$$

to give

$$\sum_{i=1}^N\left[y_i-g(\omega^Tx_i)\right]^2\approx\sum_{i=1}^Ng'(\omega_{\text{old}}^Tx_i)^2\left[\left(\omega_{\text{old}}^Tx_i+\frac{y_i-g(\omega_{\text{old}}^Tx_i)}{g'(\omega_{\text{old}}^Tx_i)}\right)-\omega^Tx_i\right]^2$$

To minimize the right-hand side, we carry out a least squares regession with target $\omega_{\text{old}}^Tx_i+(y_i-g(\omega_{\text{old}}))/g'(\omega_{\text{old}}x_i)$ on the input $x_i$, with weights $g'(\omega_{\text{old}}^Tx_i)^2$ and no intercept (bias) term. This produces the updated coefficient vector $\omega_{\text{new}}$.

These two steps, estimation of $g$ and $\omega$, are iterated until convergence. With more than one term in the PPR model, the model is built in a forward stage-wise manner, adding a pair $(\omega_m,g_m)$ at each stage.

There are a number of implementation details.

 - Although any smoothing method can in principle be used, it is convenient if the method provides derivatives. Local regression and smoothing splines are convenient.
 - After each step the $g_m$'s from previous steps can be readjusted using the backfitting procedure described in Chapter 9. While this may lead ultimatley to fewer terms, it is not clear whether it imporves prediction performance.
 - Usually the $\omega_m$ are note readjusted (partly to avoid excessive computation), although in principle they could be as well.
 - The number of terms $M$ is usually estimated as part of the forward stage-wise strategy. The model building stops when the next erm does not appreciably improve hte fit of the model. Cross-validation can also be used to determine $M$.

There are many other applications, such as density estimation, where the projection pursuit idea can be used. In particular, see the discussion of ICA in Section 14.7 and its relationship with explanatory projection pursuit. However, the projection pursuit regeression has not been widely used in the field of statistics, perhaps because at the time of introduction (1981), its computational demands exceeded the capabilities of most readily available computers. But it does not represent an important intellectual advance, one that has blossomed in its reincarnation in the field of neural networks.

### **11.3 - Neural Networks**

Here we describe the most widely used "vanilla" neural net, sometimes called the single hidden layer back-propagation network, or single layer perceptron. There has been a great deal of hyper surroundign neural networks, making them seem magical and mysterious. As we make clear in this section, they are just nonlinear statistical models, much like the projection pursuit regression model.

A neural network is a two-stage regression or classification model, typically represented by a *network diagram*. This network applies both to regression or classification. For regression, typically $K=1$ and there is only one output unit $Y_1$ at the top. However, these networks can handle multiple quantitative responses in seamless fashion, so we will deal with the general case.

For $K$-class classification, there are $K$ units at the top, with the $k$th unit modeling the probabilty of class $k$. There are $K$ units at the top, with the $k$th unit modeling the probability of class $k$. There are $K$ target measurements $Y_k$, $k=1,\ldots,K$, each being coded as $0$-$1$ variable for the $k$th class.

Derived features $Z_m$ are created from linear combinations of the inputs, and then the target $Y_k$ is modeled as a function linear combinations of the $Z_m$,


$$\begin{align*}
Z_m=\sigma(\alpha_{0m}+\alpha_m^TX),\ m=1,\ldots,M,\\
T_k=\beta_{0k}+\beta_K^TZ,\ k=1,\ldots,K,\\
f_k(X)=g_k(T),\ k=1,\ldots,K\\
\end{align*}\tag{11.5}$$

where $Z=(Z_1,Z_2,\ldots,Z_M)$, and $T=(T_1,T_2,\ldots,T_K)$.

The activation function $\sigma(v)$ is usually chosen to be the *sigmoid* $\sigma(v)=1/(1+e^{-v})$;. Sometimes Gaussian radial basis functions (Chapter 6) are used for the $\sigma(v)$, producing what is known as a *radial basis function network*.

Neural network diagrams are sometimes drawn with an additional *bias* unit feeding into every unit in the hidden and output layers. Thinking of the constant "1" as an additional input feature, this bias unit capture the intercepts $\alpha_{0m}$ and $\beta_{0k}$ in model (11.5).

The output function $g_k(T)$ allows a final transformation of the vector of outputs $T$. For regression we typically choose the identity function $g_k(T)=T_k$. Early work in $K$-class classification also used the identity function, but this was later abandoned in favor of the *softmax* function

$$g_k(T)=\frac{e^{T_k}}{\sum_{\ell=1}}^Ke{T_{\ell}}\tag{11.6}$$

This is of course exactly the transformation used in the multilogit model (Section 4.4), and produces psoitive estimates that sum to one. In Section 4.2 we discuss other problems with linear activation functions, in particular ptoentially sever masking effects.

The units in the middle of the network, computing the derived features $Z_m$, are called *hidden units* because the values $Z_m$ are not directly observed. In gerneal there can be more than one hidden layer. We can think of the $Z_m$ as a basis expansion of the original inputs $X$; the neural network is then a standard linear model, or linear multilogit model, using these transformations as inputs. There is, however, an important enhancement over the basis expansion techniques discussed in Chapter 5; here the parameters of the basis functions are learned from the data.

Notice that if $\sigma$ is the identity function, then the entire model collapses to a linear model in the inputs. Hence a neural network can be though of asa nonlinear genralization of the linear model, both for regression and classification. By introducing the nonlinear transformations $\sigma$, it greatly enlarges the class of linear models. In the below figure, we see that the rate of activation of the sigmoid depends on the norm of $\alpha_m$, and if $||\alpha_m||$ is very small, the unit will indeed be operating in the *linear part* of its activation function.


In [9]:
# latex_helper()

v = np.linspace(-10, 10, 300)
sigma = lambda x: 1 / (1 + np.exp(-x))
sigma_v = sigma(v)

s_1_2 = sigma((1 / 2) * v)
s_10 = sigma(10 * v)

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=v,
        y=sigma_v,
        mode='lines',
        line=dict(
            color='red'
        ),
        name='$v$'
    )
)

fig.add_trace(
    go.Scatter(
        x=v,
        y=s_1_2,
        mode='lines',
        line=dict(
            color='blue',
            dash='dash'
        ),
        name='$\\frac{1}{2}\\cdot v$'
    )
)

fig.add_trace(
    go.Scatter(
        x=v,
        y=s_10,
        mode='lines',
        line=dict(
            color='purple',
            dash='dash'
        ),
        name='$10\\cdot v$'
    )
)

fig.update_layout(
    xaxis=dict(
        title='$v$'
    ),
    yaxis=dict(
        title='$\\frac{1}{1+e^{-v})}$'
    )
)

fig.show()

Notice also that the neural netwrok model with one hidden layer has exactly the same form sas the projection pursuit model. The difference is that the PPR model uses nonparametric functions $g_m(v)$, while the neural network uses a far simpler function based on $\sigma(v)$, with three free parameters in its argument. In detail, viewing the neural network model as a PPR model, we identify

$$\begin{align*}
g_m(\omega_m^TX)&=&\beta_m\sigma(\alpha_{0m}+\alpha_m^TX)\\
&=&\beta_m\sigma(\alpha_{0m}+||\alpha_m||(\omega_m^TX))\tag{11.7}\\
\end{align*}$$

where $\omega_m=\alpha_m/||\alpha_m||$ i sthe $m$th unit-vector. Since $\sigma_{\beta,\alpha_{0},s}(v)=\beta\sigma(\alpha_0+sv)$ has a lower complexity than a more genreal nonparametric $g(v)$, it is not surprising that a neural network might use $20$ or $100$ such functions, while the PPR model typically uses fewer terms ($M=5$ or $10$, for example).

Finally, we note that the name "neural networks" derives from the fact that they were first developed as models for the human brain. Each unit represents a neuron, and the connecitons represent synapses. IN early models, the neurons fired when the total signal passed to that unit exceeded a certain threshold. In the model, this corresponds ot use of a step function for $\sigma(Z)$ and $g_m(T)$. Later the neural network was recognized as a useful tool for nonlinear statistical modeling, and for this purpose the step function is not smooth enough for optimization. Hence the step function was replaced by a smoother threshold function, the sigmoid.

### **11.4 - Fitting Neural Networks**

The neural network model has unknown parameters, often called *weights*, and we seek values for them that make the model fit the training data well. We denote the complete set of weights by $\mathbf{\theta}$, which consists of 

$$\begin{align*}
\{\alpha_{0m},\alpha_m;\ m=1,2,\ldots,M\}\ M(p+1)\ \text{weights}\\
\{\beta_{0k},\beta_k;\ k=1,2,\ldots,M\}\ K(M+1)\ \text{weights}\\
\end{align*}\tag{11.8}$$

For regression, we use sum-of-squared errors as our measure of fit (error function)

$$R(\mathbf{\theta})=\sum_{k=1}^K\sum_{i=1}^N(y_{ik}-f_k(x_i))^2\tag{11.9}$$

For classification we use either squared error or cross-entropy (deviance):

$$R(\mathbf{\theta})=-\sum_{i=1}^N\sum_{k=1}^Ky_{ik}\log f_k(x_i)\tag{11.10}$$

and the corresponding classifier is $G(x)=\arg\max_kf_k(x)$. With the softmax activation function and the cross-entropy error function, the neural network model is exactly a linear logistic regression model in the hidden units, and all the parameters are estimated by maximum likelihood.

Typically we don't want the global minimizer of $R(\theta)$, as this is likely to be an overfit solution. Instead some regularization is needed: this is achieved directly through a penalty term, or indirectly by early stopping. 

The generic approach to minimizing $R(\theta)$ is by gradient descent, called *back-propogation* in this setting. Because of the compositional form of the model, the garient can be easily derived usng the chain rule for differentiation. This can be computed by a forward and backward sweep over the network, keeping track only of quantitiies local to each unit.

Here is back-propagation in detail for squareed error loss. let $z_{mi}=\sigma(\alpha_{0m}+\alpha_m^Tx_i)$, from (11.5) and let $z_i(z_{1i}, z_{2i},\ldots,z_{Mi})$. Then we have

$$\begin{align*}
R(\theta)&=&\sum_{i=1}^NR_i\\
&=&\sum_{i=1}^N\sum_{k=1}^K(y_{ik}-f_k(x_i))^2\tag{11.11}\\
\end{align*}$$

with derivatives

$$\begin{align*}
\frac{\partial R_i}{\partial\beta_{km}}&=&-2(y_{ik}-f_k(x_i))g_k'(\beta_k^Tz_i)z_{mi}\\
\frac{\partial R_i}{\partial\alpha_{m\ell}}&=&-\sum_{k=1}^K2(y_{ik}-f_k(x_i))g_k'(\beta_k^Tz_i)\beta_{km}\sigma'(\alpha_m^Tx_i)x_{i\ell}
\end{align*}\tag{11.12}$$

Given these derivatives, a gradient descent update at the $(r+1)$st iteration has the form

$$\begin{align*}
\beta_{km}^{(r+1)}=\beta_{km}^{(r)}-\gamma_r\sum_{i=1}^N\frac{\partial R_i}{\partial\beta_{km}^{(r)}}\\
\alpha_{m\ell}^{(r+1)}=\alpha_{m\ell}^{(r)}-\gamma_r\sum_{i=1}^N\frac{\partial R_i}{\partial\alpha_{m\ell}^{(r)}}\\
\end{align*}\tag{11.13}$$

where $\gamma_r$ is the *learning rate*, discussed below, 

Now write (11.12) as

$$\begin{align*}
\frac{\partial R_i}{\partial\beta_{km}}=\delta_{ki}z_{mi}\\
\frac{\partial R_i}{\partial\alpha_{m\ell}}=s_{mi}x_{i\ell}\\
\end{align*}\tag{11.14}$$

The quantities $\delta_{ki}$ and $s_{mi}$ are "errors" from the current model at the output and hidden layer units, respectively. From their definitions, these errors satisfy

$$s_{mi}=\sigma'(\alpha_m^Tx_i)\sum_{k=1}^K\beta_{km}\delta_{ki}\tag{11.15}$$

known as the *back-propogation equations*. Using this, the updates in (11.13) can be implemented with a two-pass algorithm. In the *forward pass*, the current weights are fixed and the predicted values $f_k(x_i)$ are computed from formula (11.5). In the *backward pass*, the errors $\delta_{ki}$ are computed, and then back-propagated via (11.15) to give the erros $s_{mi}$. Both sets of errors are then used to compute the gradients for the updates in (11.13), via (11.14).

This two-pass procedure is what is known as back-propagation. It has also been called the *delta rule*. The computational compnents for cross-entropy have the same form as those for the sum of squares error function.

The advantages of back-propagation are its simple, local nature. In the back propagation algorithm, each hidden unit passes and receiives information only to and from units that share a connection. Hence it can be ipmlmented efficiently on a parallel architectuyre  computer. 

The updates in (11.13) are a kind of *batch learning*, with the parameter upates being a sum over all of the training cases. LEarning cal aslo be carried out online--processing each observation one at a time, updating the gradient after each training case, and cycling through the training cases many times. IN this case, the sums in equations (11.13) are replaced by a single summand. A *training epoch* refers to one sweep through entire training set. ONline training allows the network to handle ver y large training sets, and also to update the weights as new observations come in.

The learning rate $\gamma_r$ for batch learning is usually taken to be a constant, and can also be optimized by a line search that minizes the eror function at each upadte. With online learning $\gamma_r$ should decrease to zero as the iteration $r\rightarrow\infty$. THis learning is a form of *stochastic approximation*; results in this field ensure convergence if $\gamma_r\rightarrow0$, $\sum_r\gamma_r=\infty$, and $\sum_r\gamma_r^2<\infty$.

Back-propagation can be very slow, and for that reason is usually not the method of choice. Second-order techniques such as Newton's method are not attractive here, because the second derivative matrix of $R$ (the Hessian) can be very large. Better approaches to fitting include conjugate gradients and variable metric methods. These avoid explicit computation of the second derivative matrix while still provider faster convergence.

### **11.5 Some Issues in Training Neural Networks**

There is quite an art in training neural networks. The model is generally overparametrized, and the optimization problem is nonvonvex and unstable unless certain guidelines are followed.

#### **11.5.1 - Starting Values**

Note that if the weights are near zero, then the operativ epart of the sigmoid is roughly linear, and hence the neural network collapses into an approximately linea rmodel. Usually starting values for weights are chosen to be random values near zero. Hence the model starts out linear, and beomces nonlinear as the wiehgt increase. Individual units localize to directions and introduce nonlinearities where needed. Use of exact zero weights leads to zero derivatives and perfect symmetry, and the algorithm never moves. Starting instead with large weights often leads to poor solutions.

#### **11.5.2 - Overfitting**

Often neural networks have too many weights and will overfit the data at the global minimum of $R$. In early developments of neural networks, either by design or by accidnet, an early stopping rule was used to avoid overfitting. Here we trianing the model only for a while, and stop well before we approach the global minimum. SInce the wieghts start at a highly-regularized (linear) solution, this has the effect of shrinking the final model toward a linear model. A validation dataset is useful for determining when to stop, since we expect the validation error to start increasing.

A mroe explicit method for regularization is *weight decay*, which is analogous to ridge regression used for linear models (Section 3.4.1). We add a penalty to the erro function $R(\theta)+\lambda J(\theta)# where

$$J(\theta)=\sum_{k,m}\beta_{km}^2+\sum{m,\ell}\alpha_{m\ell}^2\tag{11.16}$$

and $\lambda\ge0$ is a tuning parameter. Larger values of $\lambda$ will tend to shrink the weights toward zero: typically cross-validation is used to estimate $\lambda$. The effect of the penalty is to simply add terms $2\beta_{km}$ and $2\alpha_{m\ell}$ to the respective gradient expression (11.13). Other forms for the penalty have been proposed, for example,

$$J(\theta)=\sum_{k,m}\frac{\beta_{km}^2}{1+\beta_{km}^2}+\sum_{m,\ell}\frac{\alpha_{m\ell}^2}{1+\alpha_{m\ell}^2}\tag{11.17}$$

known as the *weight elimination penalty*. This has the effect of shrinking smaller wieghts more than (11.16) does.

#### **11.5.3 - Scaling of the Inputs**

Since the scaling of the inputs determines the effective scaling of the weights in the bootom layer, it can have a alrge effect on the quality of the final solution. At the outset it is best to standardize al linputs to have mean zero and standard devation one. THis ensures all inputs are treated equally in the regularization process, and allows one to choose a meaningful range for the random starting weights. With standardized inputs, it is typical to take random uniform weights over the range $[-0.7,+0.7]$.

## **13 - Prototype Methods and Nearest-Neighbors**

### **13.1 - Introduction**

In this chapter, we discuss some simple and esentially model-free methods for classification and pattern recognition. Because they are highly unstrucutred, they typically are not useful for understanding the nature of the relationship between the features and class outcome. However, as *black box* prediction engines, they can be very effective, and are often among the best performers in real data problems. 

### **13.2 - Prototype Methods**

Throughout this chapter, our training data consists of the $N$ pairs $(x_1,g_1),\ldots,(x_n,g_N)$ where $g_i$ is a class label taking values $\{1,2,\ldots,K\}$. Prototype methods represent the training data by a set of points in feature space. These prototypes are typically not examples from the training sample except in the case of $1$-nearest-neighbor classification.

Each prototype has an associated class label, and classification of a query point $x$ is made ot hte classo f the closest prototype. "Closes" is usually defined by Euclidean distance int he feature space, after each feature has been standardized to have overall mean $0$ and variance $1$ in the training sample. Euclidean distance is appropriate for quantitative features. We discuss distance measures between qualitative and other features values in Chapter 14.

Let our training data consist of $N$ pairs $(x_1,g_1),\ldots,(x_n,g_n)$ where $g_i$ is a class label taking values in $\{1,2,\ldots,K\}$. Protoype methods represent the training data by a set of points in feature space. These prototypes are typically not examples from the training sample, except in the case of $1$-nearest-neighbor classification. 

Each prototype has an associated class label, and classification of a query point $x$ is made to the class of the closest protype. "Closest" is usually defined by Euclidean distance in the feature space, after each feature has been standardized to have overall mean $0$ and variance $1$ in the training sample. Euclidean distance is approximate for quantitative features. 

#### **13.2.1 - $K$-means Clustering**

$K$-means clustering is a method for finding clusters and cluster centers in a set of unlabeled data. One chooses the deired number of clusters, say $R$, and the $K$-means procedure iteratively moves the centers to minimize the total within cluster variance. Given an initial set of centers, the $K$-means algorithm alternates in two steps:

 - for each center we identify thesubset of training points (its cluster) that is closer to it that any other center;
 
 - the means of each feature for the data points in each cluster are computed, and this mean vector becomes the new center for that cluster.

These two steps are iterated until convergenc. Typically the intital centers are $R$ randomly choses observatiosn from the training data. Details of the $K$-means procedure, as well as generalizations allowing for different variable types and more genreal distance measrues, are given in Chapter 14.

To use $K$-means clustering for classification of labeled data, the steps are:

 - apply $K$-means clustering to the training data in each class separately, using $R$ prototypes per class;

 - assign a class label to each of teh $K\times R$ prototypes;

 - classify a new feature $x$ to the class of the closest prototype.

**Algorithm 13.1 - Learning Vecotr Quantization--LVQ**

1. Choose $R$ initial prototypes for each class: $m_1(k),m_2(k),\ldots,m_R(k)$, $k=1,2,\ldots,K$, for example, by sampling $R$ training points at random from each class.

2. Sample a training point $x_i$ randomly (with replacement), and let $(j,k)$ index the closest prototype $m_j(k)$ to $x_i$.

    - If $g_i=k$ (i.e. they are in the same class), move the prototype towards the training point:

    $$m_j(k)\leftarrow m_j(k)+\epsilon(x_i-m_j(k))$$

    where $\epsilon$ is the *learning rate*.

    - If $g\ne k$ (i.e., they are in different classes), move the prototype away from the start point:

    $$m_j(k)\leftarrow m_j(k)-\epsilon(x_i-m_j(k))$$

3. Repeat step 2, decreasing the learning rate $\epsilon$ with each iteration towards zero.

There is an obvious shortcoming, for each class, the other classes do not have a say in the positioning of the prototypes for that class. 

#### **13.2.2 - Learning Vector Quantization**

In this technique due to Kohonen (1989), prototypes are placed stategically with respect to the decision boundaries in an ad-hoc way. LVW is an *online algorithm*--observations are processed one at a time.

The idea is that training points attract prototypes of the correct class, and repel other prototypes. When the iteratiosn settle down, prototypes should be close to the training points in their class. The learning rate $\epsilon$ is decreased to zero with each iteration, following the guidelines for stochastic approximation learning rates (Section 11.4.)

## **14 - Unsupervised Learning**

### **14.4 - Self-Organizing Maps**

This method can be viewed as a constrained version of $K$-means clustering, in which the protypes are enouraged to lie in a one- or two-dimensional manifold in the feature space. The resulting manifold is also referred to as a *constrained topological map*, since the original high-dimensial observations can be mapped down onto the two-dimensioanl coordinate system. The orignal SOM algorithm was online--observations are procesed one at a time--and later a batch version was proposed.

We consider a SOM with a two-dimensional rectangular grid of $K$ prototypes $m_j\in\mathbb{R}^p$ (other choices, such as hexagonal grids, can also be used). Each of the $K$ protoypes are parameterized with respect to an integer coordinate pair $\ell_j\in\mathcal{Q}_1\times\mathcal{Q}_2$. Here $\mathcal{Q}_1=\{1,2,\ldots,q_1\}$, similarly $\mathcal{Q}_2$, and $K=q_1\cdot q_2$. The $m_j$ are initialized, for example, to lie in the two-dimensional principal component plane of the data. We can think of the prototypes as "buttons," "sewn" on the principal component plane in a regular pattern. The SOM procedure tries to bend the plane so that the buttons approximate the data points as well as possible. Once the model is fit, the observations can mapped down onto the two-dimensional grid. 

The observations $x_i$ are processed one at a time. We find the closest prototype $m_j$ to $x_i$ in Euclidean distance in $\mathbb{R}^p$, and then for all neighbors $m_k$ of $m_j$, move $m_k$ toward $x_i$ via the update

$$m_k\leftarrow m_k+\alpha(x_i-m_k)$$

The "neighbors" of $m_j$ are defned to be all $m_k$ such taht the distance between $\ell_j$ and $\ell_k$ is small. The simplest approach uses Euclidean distance, and "small" is determined by a threshold $r$. This neighborhood always includes the closest prototype $m_j$ itself.

Notice that distance is deifned in the space $\mathcal{Q}_1\times\mathcal{Q}_2$ of integer topological coordinates of the prototypes, rather than in the feature space $\mathcal{R}^p$. The effect of the update is to move the prototypes closer to the data, but also to maintiain a smooth two-dimensional spatial relatinoshisp between the prototypes.

The performance of the SOM algorithm depends on the learning rate $\alpha$ and the distance threshold $r$. Typicallly $\alpha$ is decreased from say $1.0$ to $0.0$ over a few thousand iterations (one per observation). Similarly, $r$ is decreased linearly from starting value $R$ to $1$ over a few thousand iterations. 

More sophisticated versions modify the update step according to distance:

$$m_k\leftarrow m_k+\alpha\mathcal{h}(||\ell_j-\ell_k||)(x_i-m_k)$$

where the *neighborhood function* $\mathcal{h}$ gives more weight to prototypes $m_k$ with indices $\ell_k$ closer to $\ell_j$ than to those further away.

If we take the distance $r$ small enough so that each neighborhood contains only one point, then the spatial conneciton between prototypes is lost. In that case one can show that the SOM algorithm is an online version of $K$-means clustering, and eventually stabilizes at one of the local minima found by $K$-means. Since the SOM is a constrained version of $K$-means clustering, it is important to check whether the constraint is reasonable in any given problem. One can do this by computing the reconstruction error $||x-m_j||^2$, summed over observations, for both methods. This will necessarily be smaller for $K$-means, but should not be much smaller if the SOM is a reasonable approximation. 